In [10]:
import json
import zipfile
import os

In [3]:
actions_mapping = {0:   'idle', 
                   1:	'approach',
                   2:	'retreat',
                   3:	'lift',
                   4:	'place',
                   5:	'hold',
                   6:	'pour',
                   7:	'cut',
                   8:	'hammer',
                   9:	'saw',
                   10:	'stir',
                   11:	'screw',
                   12:	'drink',
                   13:	'wipe'
                   }
 
objects_mapping =  {0:	'bowl',
                    1:	'knife',
                    2:	'screwdriver',
                    3:	'cutting board',
                    4:	'whisk',
                    5:	'hammer',
                    6:	'bottle',
                    7:	'cup',
                    8:	'banana',
                    9:	'cereals',
                    10:	'sponge',
                    11:	'wood',
                    12:	'saw',
                    13:	'hard drive',
                    14:	'left hand',
                    15:	'right hand'
}

In [90]:
GROUND_TRUTH_PATH = "bimacs_rgbd_data_ground_truth.zip"

def jsons_from_zip(zip_filename):
    with zipfile.ZipFile(zip_filename, mode="r") as archive:
        files = archive.namelist()

    jsons = []
    for f in files:
        with zipfile.ZipFile(zip_filename, mode="r") as archive:
            with archive.open(f) as myfile:
                jsons.append({f : json.loads(myfile.read())})
        
    return jsons

def map_actions(jsons_ground_truth):
    for j_file in jsons_ground_truth:
        r = list(j_file.values())[0]['right_hand']
        l = list(j_file.values())[0]['left_hand']

        for i in range (len(r)):
            if i%2 != 0 and r[i]!=None:
                r[i] =  actions_mapping[r[i]]
        
        for i in range (len(l)):
            if i%2 != 0 and l[i]!=None:
                l[i] =  actions_mapping[l[i]]


def get_frames(ground_truth_json):
    for k, v in ground_truth_json.items():
        frames = [i for i in v['right_hand'] if type(i)==int]
        actions = [i for i in v['right_hand'] if type(i)!=int]
        # frames.extend([i for i in v['left_hand'] if type(i)==int])
        frame_files = [k.replace('bimacs_rgbd_data_ground_truth', 'bimacs_derived_data').replace('.json', '/spatial_relations/') + 'frame_' + str(i) + '.json' for i in frames]
    return frame_files, actions

def read_spacial_relations(jsons_ground_truth):
    all_sequences = []
    for json_1 in jsons_ground_truth:
        frame_files, actions = get_frames(json_1)

        seq_1 = {}
        for path in frame_files:
            components = path.split('/')
            seq_1['task'] = components[2]
            seq_1['subject'] = components[1]
            seq_1['take'] = components[3]
            with open(path) as f:
                d = json.load(f)
                seq_1[components[-1].replace('.json', '')] = {}
                if frame_files.index(path) < len(actions):
                    seq_1[components[-1].replace('.json', '')]['action'] = actions[frame_files.index(path)]

                seq_1[components[-1].replace('.json', '')]['spectal_relations'] = []
                for js in d:
                    if js['relation_name'] in ('moving together', 'halting together', 'fixed moving together', 
                        'getting close', 'moving apart', 'temporal') and js['object_index'] < js['subject_index']:
                        seq_1[components[-1].replace('.json', '')]['spectal_relations'].append({'relation_name' : js['relation_name'],
                                                                    'objects' : [objects_mapping[js['object_index']], objects_mapping[js['subject_index']]]})
                        # seq_1[components[-1].replace('.json', '')]['relation_name'] = js['relation_name'] 
                        # seq_1[components[-1].replace('.json', '')]['objects'] = [objects_mapping[js['object_index']], objects_mapping[js['subject_index']]]
                    
        all_sequences.append(seq_1)
    return all_sequences

def save_jsons_data(all_sequences, filename):
    save_file = open(filename, "w")  
    json.dump(all_sequences, save_file, indent = 6)  
    save_file.close()  

In [91]:
jsons_ground_truth = jsons_from_zip(GROUND_TRUTH_PATH)
map_actions(jsons_ground_truth) # action ids are translated into actions
all_sequences = read_spacial_relations(jsons_ground_truth)

Done
Done


In [92]:
all_sequences

[{'task': 'task_6_w_hard_drive',
  'subject': 'subject_6',
  'take': 'take_8',
  'frame_0': {'action': 'idle', 'spectal_relations': []},
  'frame_57': {'action': 'approach',
   'spectal_relations': [{'relation_name': 'getting close',
     'objects': ['bowl', 'cutting board']},
    {'relation_name': 'getting close', 'objects': ['knife', 'cutting board']},
    {'relation_name': 'getting close',
     'objects': ['screwdriver', 'cutting board']},
    {'relation_name': 'getting close',
     'objects': ['cutting board', 'whisk']}]},
  'frame_82': {'action': 'lift', 'spectal_relations': []},
  'frame_101': {'action': 'screw',
   'spectal_relations': [{'relation_name': 'moving apart',
     'objects': ['bowl', 'screwdriver']},
    {'relation_name': 'moving apart',
     'objects': ['screwdriver', 'cutting board']},
    {'relation_name': 'fixed moving together', 'objects': ['bowl', 'whisk']},
    {'relation_name': 'moving apart', 'objects': ['cutting board', 'whisk']},
    {'relation_name': 'movi

In [94]:
save_jsons_data(all_sequences, "all_sequences_1.json")